# Building Model Analysis

In [6]:
# import the libraries
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import tensorflow as tf
import pandas as pd
from datetime import datetime
print("TensorFlow version: ",tf.__version__)  #print the version of tensorflow

TensorFlow version:  2.1.0


In [2]:
#Format into dates
def format_time(unix_timestamp):
    format = '%Y-%m-%d'
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime(format)

#Create list of datetime objects from timestamps
def gen_datetimes(li):
    datetimes = []
    for time in li:
        datetimes.append(datetime.utcfromtimestamp(time))
    return datetimes

def get_weekday2(year, month, day):
    dates = pd.DataFrame()
    dates['y'] = year
    dates['m'] = month
    dates['d'] = day
    dates['dates'] = dates['y'].astype('str') + '-' + dates['m'].astype('str') + '-' + dates['d'].astype('str')
    return get_weekday(dates['dates'])

#Get day of week based on date
def get_weekday(dates):
    return [1 if (datetime.strptime(d,"%Y-%m-%d").weekday() >= 5) else 0 for d in dates]

In [3]:
def clean_data(location, skiprows = 0):
    df_weather = pd.read_csv(location, skiprows=skiprows)
    df_weather = df_weather.drop(columns=['Hour', 'Minute'])
    df_weather = df_weather[df_weather.DNI != 0]

    #Take mean, max and min for each DNI in DataFrame and mean for everything else
    
    max_dni = df_weather.groupby(['Year', 'Month', 'Day']).max().reset_index()['DNI']
    min_dni = df_weather.groupby(['Year', 'Month', 'Day']).min().reset_index()['DNI']
    
    df_weather = pd.DataFrame(df_weather.groupby(['Year', 'Month', 'Day']).mean().reset_index())
    
    df_weather.insert(4, 'DNI Max', max_dni)
    df_weather.insert(5, 'DNI Min', min_dni)
    
    return df_weather

In [34]:
def training_prep(data, square_feet, building_type):
    data = data.loc[:, ['Year', 'Month', 'Day', 'DNI', 'DNI Max', 'DNI Min', 'Wind Speed', 'Precipitable Water', 'Wind Direction', 'Relative Humidity', 'Temperature', 'Pressure']]
    data.loc[:, 'Weekday'] = get_weekday2(data['Year'], data['Month'], data['Day'])
    data.loc[:, 'Square Feet'] = square_feet
    data.loc[:, 'Type'] = building_type
    return data

In [37]:
#Taking data for University of Michigan NUCLEAR ENGINEERING LABORATORY
michigan_data = training_prep(clean_data('Datasets/AnnArbor_Weather.csv'), 20500, 2)
michigan_data.head()

,Year,Month,Day,DNI,DNI Max,DNI Min,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,Weekday,Square Feet,Type
0,2019,1,1,52.111111,178,3,0.377778,1.033333,295.222222,86.873333,4.133333,993.888889,0,20500,2
1,2019,1,2,396.000000,751,17,0.180000,0.400000,281.500000,50.143000,-1.500000,1011.400000,0,20500,2
2,2019,1,3,299.700000,752,7,0.240000,0.740000,290.400000,83.652000,1.970000,997.400000,0,20500,2
3,2019,1,4,281.900000,576,34,0.190000,1.240000,221.000000,83.261000,1.970000,997.300000,0,20500,2
4,2019,1,5,10.000000,21,3,0.200000,2.266667,177.666667,94.240000,2.000000,986.000000,1,20500,2
